In [ ]:
import fsspec
import xarray as xr
import dask
import intake
import numpy as np
import matplotlib.pyplot as plt
import regionmask

dask.config.set(**{"array.slicing.split_large_chunks": False})

SQM_PER_HA = 10000
cat = intake.open_catalog("./catalog.yaml")


def compute_grid_area(da):
    R = 6.371e6
    total_area = 4 * np.pi * R**2
    dϕ = np.radians((da["lat"][1] - da["lat"][0]).values)
    dλ = np.radians((da["lon"][1] - da["lon"][0]).values)
    dA = R**2 * np.abs(dϕ * dλ) * np.cos(np.radians(da["lat"]))
    areacella = dA * (0 * da + 1)

    return areacella / SQM_PER_HA

In [ ]:
lat = "00N"
lon = "070W"
da_zarin = (
    cat.hansen_emissions_px(lat=lat, lon=lon)
    .to_dask()
    .rename({"x": "lon", "y": "lat"})
    .coarsen(lon=100, lat=100)
    .sum()
    .load()
)

In [ ]:
mapper = fsspec.get_mapper(f"gs://carbonplan-scratch/global-forest-emissions/{lat}_{lon}.zarr")
da_americas = xr.open_zarr(mapper)["emissions"]
da_area = compute_grid_area(da_americas.isel(year=0, drop=True))
da_americas = (da_americas * da_area).coarsen(lat=100, lon=100).sum().load()

In [ ]:
da_americas_sum = da_americas.sum("year")

In [ ]:
da_zarin.where(da_zarin).plot(vmin=0, vmax=200000)
plt.title("Zarin Emissions")

In [ ]:
da_americas_sum.where(da_americas_sum).plot(vmin=0, vmax=200000)
plt.title("CarbonPlan Emissions")

In [ ]:
((da_zarin - da_americas_sum) / da_zarin).plot(robust=True)
plt.title("% difference")

In [ ]:
coarsened_url = f"gs://carbonplan-scratch/global-forest-emissions/americas/3000m/raster_v2.zarr"
mapper = fsspec.get_mapper(coarsened_url)
ds_ameri = xr.open_zarr(mapper)

In [ ]:
ds_ameri

In [ ]:
ds_ameri.isel(year=0, lat=slice(None, None, 10), lon=slice(None, None, 10)).emissions.plot(
    robust=True
)

In [ ]:
countries = regionmask.defined_regions.natural_earth.countries_110
names = dict(zip(countries.numbers, countries.names))
mask = countries.mask(ds_ameri.emissions)

df = ds_ameri.emissions.groupby(mask).sum().to_pandas()
columns = {k: names[int(k)] for k in df.columns}
df = df.rename(columns=columns)
df.head()

In [ ]:
for k, s in df.items():
    plt.figure()
    (s / 1e9).plot()
    plt.title(k)

In [ ]:
data = (df / 1e9).to_dict()

In [ ]:
out = {}
for k, d in data.items():
    l = []
    for y, v in d.items():
        l.append({"year": y, "emissions": v})
    out[k] = l

In [ ]:
import json

with open("data.json", "w") as f:
    json.dump(out, f, indent=2)